# Column Generation

In [2]:
import itertools
import pandas as pd

In [3]:
# Each row is a person's unavailability in EST time: [personal commitments, morning break, afternoon break, lunch break]
chairs_est_unavailability = [
    ["M/W 13:00 to 15:00", "M/T/W/Th/F 12:20 to 12:40", "M/T/W/Th/F 17:20 to 17:40", "M/T/W/Th/F 15:20 to 16:00"],
    ["T/Th 16:00 to 18:00", "M/T/W/Th/F 11:20 to 11:40", "M/T/W/Th/F 16:20 to 16:40", "M/T/W/Th/F 14:20 to 15:00"],
    ["M/W 13:00 to 15:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/F 9:00 to 11:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["Th 11:00 to 15:00", "M/T/W/Th/F 12:20 to 12:40", "M/T/W/Th/F 17:20 to 17:40", "M/T/W/Th/F 15:20 to 16:00"],
    ["W/F 10:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/Th 16:00 to 18:00", "M/T/W/Th/F 12:20 to 12:40", "M/T/W/Th/F 17:20 to 17:40", "M/T/W/Th/F 15:20 to 16:00"],
    ["Th 8:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/F 10:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/F 11:00 to 14:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"]
]
members_est_unavailability = [
    ["T/Th 14:00 to 16:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/W 13:00 to 15:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/F 9:00 to 11:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/Th 11:00 to 13:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/F 14:00 to 16:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/F 17:00 to 19:00", "M/T/W/Th/F 11:20 to 11:40", "M/T/W/Th/F 16:20 to 16:40", "M/T/W/Th/F 14:20 to 15:00"],
    ["M 12:00 to 14:00", "M/T/W/Th/F 8:20 to 8:40", "M/T/W/Th/F 13:20 to 13:40", "M/T/W/Th/F 11:20 to 12:00"],
    ["M/W 11:00 to 13:00", "M/T/W/Th/F 10:20 to 10:40", "M/T/W/Th/F 15:20 to 15:40", "M/T/W/Th/F 13:20 to 14:00"],
    ["Th 8:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["F 11:00 to 13:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/Th 10:00 to 12:00", "M/T/W/Th/F 8:20 to 8:40", "M/T/W/Th/F 13:20 to 13:40", "M/T/W/Th/F 11:20 to 12:00"],
    ["M/F 14:00 to 16:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/F 15:00 to 17:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["W 12:00 to 17:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["W/F 10:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/Th 13:00 to 15:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/W 10:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/F 10:00 to 12:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["Th 10:00 to 15:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/Th 15:00 to 17:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/T 8:00 to 10:00", "M/T/W/Th/F 4:20 to 4:40", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 7:20 to 8:00"],
    ["Th 11:00 to 15:00", "M/T/W/Th/F 12:20 to 12:40", "M/T/W/Th/F 17:20 to 17:40", "M/T/W/Th/F 15:20 to 16:00"],
    ["M/T 12:00 to 14:00", "M/T/W/Th/F 11:20 to 11:40", "M/T/W/Th/F 16:20 to 16:40", "M/T/W/Th/F 14:20 to 15:00"],
    ["T/W 17:00 to 19:00", "M/T/W/Th/F 11:20 to 11:40", "M/T/W/Th/F 16:20 to 16:40", "M/T/W/Th/F 14:20 to 15:00"],
    ["T/W 15:00 to 17:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["M/W 12:00 to 14:00", "M/T/W/Th/F 11:20 to 11:40", "M/T/W/Th/F 16:20 to 16:40", "M/T/W/Th/F 14:20 to 15:00"],
    ["M/F 11:00 to 14:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
    ["T/Th 6:00 to 8:00", "M/T/W/Th/F 2:20 to 2:40", "M/T/W/Th/F 7:20 to 7:40", "M/T/W/Th/F 5:20 to 6:00"],
    ["F 13:00 to 17:00", "M/T/W/Th/F 10:20 to 10:40", "M/T/W/Th/F 15:20 to 15:40", "M/T/W/Th/F 13:20 to 14:00"],
    ["T/Th 8:00 to 10:00", "M/T/W/Th/F 9:20 to 9:40", "M/T/W/Th/F 14:20 to 14:40", "M/T/W/Th/F 12:20 to 13:00"],
]

group1_est_unavailability = [*[chairs_est_unavailability[i-1] for i in [1,5,7,2,3]],*[members_est_unavailability[i-1] for i in [22,6,24,26,23,8,29,1,2,3,4,5,9,10,12]]]
group2_est_unavailability = [*[chairs_est_unavailability[i-1] for i in [4,6,8,9,10]],*[members_est_unavailability[i-1] for i in [13,14,15,16,17,18,19,20,25,27,30,7,11,21,28]]]

In [4]:
# Monday    =   1 -  34
# Tuesday   =  35 -  68
# Wednesday =  69 - 102
# Thursday  = 103 - 136
# Friday    = 137 - 170
days_slot_start = {
    "M": 0,
    "T": 34,
    "W": 68,
    "Th": 102,
    "F": 136,
}

def calculate_unavailability(unavailable_times):
  overall_busy = []
  for busy_person in unavailable_times:
    person_busyness = []
    for busy_time in busy_person:
      days_and_time = busy_time.split(" ")
      days = days_and_time[0].split("/")
      start_time = days_and_time[1].split(":")
      end_time = days_and_time[3].split(":")
      start_minutes = int(start_time[0]) * 60 + int(start_time[1])
      end_minutes = int(end_time[0]) * 60 + int(end_time[1])

      # 8am to 7:20pm in minutes
      day_start = 8 * 60
      day_end = 19 * 60 + 20

      # Ensure times are within the available day
      start_minutes = max(start_minutes, day_start)
      end_minutes = min(end_minutes, day_end)

      # Calculate unavailability
      slots = []
      for slot_num in range(1, 35):
        slot_start = (slot_num - 1) * 20 + day_start
        slot_end = slot_num * 20 + day_start

        if slot_start >= end_minutes:
          break
        if slot_end <= start_minutes:
          continue
        slot_start = max(slot_start, start_minutes)
        slot_end = min(slot_end, end_minutes)

        # Add unavailability for all days
        for d in days:
          slot_i = days_slot_start[d]
          slots.append((slot_num + slot_i))
      person_busyness.append(slots)

    flattened_unavailability = [timeslot for sublist in person_busyness for timeslot in sublist]
    overall_busy.append(list(set(flattened_unavailability)))
  return overall_busy

group1_person_unavailability = calculate_unavailability(group1_est_unavailability)
group2_person_unavailability = calculate_unavailability(group2_est_unavailability)

In [5]:
# Maps which members are unavailable for each timeslot
group1_unavailability_mapping = {key: [] for key in range(1, 171)}
for i, person_times in enumerate(group1_person_unavailability):
  for timeslot in person_times:
    group1_unavailability_mapping[timeslot].append(i)

print(group1_unavailability_mapping)

group2_unavailability_mapping = {key: [] for key in range(1, 171)}
for i, person_times in enumerate(group2_person_unavailability):
  for timeslot in person_times:
    group2_unavailability_mapping[timeslot].append(i)

print(group2_unavailability_mapping)

{1: [], 2: [], 3: [], 4: [], 5: [4, 12, 13, 14, 15, 16, 17, 18, 19], 6: [], 7: [], 8: [10, 11], 9: [], 10: [10, 15], 11: [3, 6, 7, 8, 9, 10, 15], 12: [10, 15], 13: [8, 9, 10, 15], 14: [0, 1, 2, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19], 15: [4, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19], 16: [0, 4, 8, 9, 13], 17: [0, 4, 8, 9, 10, 11, 13], 18: [0, 4, 8, 9, 10, 11, 13], 19: [0, 4, 13, 16, 19], 20: [0, 3, 4, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19], 21: [0, 3, 4, 6, 7, 8, 9, 13, 16, 19], 22: [16, 19], 23: [0, 1, 2, 5, 10, 11, 16, 19], 24: [0, 1, 2, 5, 16, 19], 25: [], 26: [3, 6, 7, 8, 9], 27: [], 28: [6], 29: [0, 1, 2, 5, 6], 30: [6], 31: [6], 32: [6], 33: [6], 34: [], 35: [], 36: [], 37: [], 38: [14], 39: [4, 12, 13, 14, 15, 16, 17, 18, 19], 40: [14], 41: [14], 42: [10, 11, 14], 43: [14], 44: [], 45: [3, 6, 7, 8, 9], 46: [], 47: [9], 48: [0, 1, 2, 4, 5, 9, 12, 13, 14, 15, 16, 17, 18, 19], 49: [4, 9, 12, 13, 14, 15, 16, 17, 18, 19], 50: [9], 51: [9, 10, 11], 52: [9, 10, 11], 53: [

In [6]:
num_people = 20
combinations = list(itertools.product([0, 1], repeat=num_people))

# Infeasible columns criteria
max_people_in_meetings = 12
total_sums_allowed = [0,6, 12]
members_i_start = 5  # Corresponds to member 1 in list (6th element).
member_to_chair_ratio = 5  # 5 members per 1 chair.

# Filters Applied:
#   1. At most 12 people can be in meetings at a time for each subgroup.
#   2. Each room requires 6 attendees.
#   3. A 1:5 ratio between chairs and members must be respected.
generic_feasible_combinations = [
    combo for combo in combinations
    if sum(combo) in total_sums_allowed and sum(combo) <= max_people_in_meetings
    and sum(combo[:members_i_start]) * member_to_chair_ratio == sum(combo[members_i_start:])
]

print("Total possible combinations:", len(combinations))
print("Total feasible combinations:", len(generic_feasible_combinations))

Total possible combinations: 1048576
Total feasible combinations: 45046


In [7]:
# Filter applied ensures unavailable individuals are not assigned meetings.
group1_feasible_combinations = [
    [
        combo for combo in generic_feasible_combinations
        if all(combo[i] == 0 for i in group1_unavailability_mapping[timeslot])
    ] for timeslot in range(1, 171)
]

group2_feasible_combinations = [
    [
        combo for combo in generic_feasible_combinations
        if all(combo[i] == 0 for i in group2_unavailability_mapping[timeslot])
    ] for timeslot in range(1, 171)
]

print("Possible Number of Combinations for Group 1 Timeslots:", [len(timeslot_combo) for timeslot_combo in group1_feasible_combinations])
print("Possible Number of Combinations for Group 2 Timeslots:", [len(timeslot_combo) for timeslot_combo in group2_feasible_combinations])

Possible Number of Combinations for Group 1 Timeslots: [45046, 45046, 45046, 45046, 85, 45046, 45046, 9296, 45046, 9296, 505, 9296, 2421, 1, 1, 2575, 760, 760, 2575, 1, 113, 9296, 506, 1651, 45046, 1915, 45046, 20021, 2861, 20021, 20021, 20021, 20021, 45046, 45046, 45046, 45046, 20021, 85, 20021, 20021, 4621, 20021, 45046, 1915, 45046, 20021, 2, 25, 20021, 4621, 4621, 20021, 1, 1015, 20021, 936, 2861, 18019, 1420, 18019, 9010, 1288, 9010, 20021, 20021, 20021, 45046, 45046, 45046, 45046, 45046, 85, 45046, 45046, 9296, 45046, 20021, 1015, 20021, 9296, 1, 5, 4720, 1420, 1420, 9010, 1, 506, 45046, 1651, 5006, 45046, 1915, 45046, 20021, 2861, 20021, 20021, 20021, 20021, 45046, 20021, 20021, 20021, 20021, 85, 20021, 20021, 4621, 20021, 3565, 169, 3565, 6865, 7, 19, 14015, 3565, 3565, 6865, 1, 379, 20021, 936, 2861, 18019, 1420, 18019, 18019, 2003, 18019, 45046, 45046, 45046, 45046, 45046, 45046, 45046, 20021, 85, 20021, 20021, 4621, 20021, 20021, 1015, 20021, 20021, 7, 85, 20021, 9296, 9296,

# Main

In [8]:
!pip install gurobipy>=9.5.1

In [9]:
import gurobipy as gp
import itertools
from gurobipy import GRB as GRB

In [10]:
#Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.setParam('Seed', 435)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Set parameter Seed to value 435
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca


In [11]:
# Sets
D = range(1,6)
T = range(1,171)
T_d = {d : (T[(d-1)*len(T)//len(D):(d)*len(T)//len(D)])for d in D}
C = range(1,6)
M = range(6,21)
I = (*C,*M)
H_t = {t : range(1,len(group1_feasible_combinations[t-1])) for t in T}

In [12]:
# penalties
# separated to make them easier to read
# taken from this graph: https://www.desmos.com/calculator/g5rhs6n2ke
penalties = [(0,0), (4,4), (8, 11.31), (12, 20.78), (16, 32), (34, 99.13)]
Q = range(1,len(penalties)+1)

In [13]:
# Parameters

# need to sort this one out with column generation

a = group1_feasible_combinations
f = [[sum(a[t-1][h-1]) // 6 for h in H_t[t]] for t in T]
n = 8 # Need to inpute
u = {q : penalties[q-1][1] for q in Q}
v = {q : penalties[q-1][0] for q in Q}
w = .001 # Need to inpute

In [14]:
indices = [(t, h) for t in T for h in H_t[t]]

In [15]:
# Decision Varbiables
alpha = model.addVars(indices, vtype=GRB.BINARY, name="alpha")
y = model.addVars(I, D, Q, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name = "y")

In [16]:
for i in I:
    for d in D:
        model.addSOS(GRB.SOS_TYPE2, [y[i,d,q] for q in Q])

In [17]:
# Objective Value
model.setObjective(sum(f[t-1][h-1]*alpha[t,h] for t in T for h in H_t[t]) - w * sum(u[q] * y[i,d,q] for i in I for d in D for q in Q),sense=GRB.MAXIMIZE)

In [18]:
# Constraints
model.addConstrs(sum(alpha[t,h] for h in H_t[t]) <= 1 for t in T)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) <= n for i in I for d in D)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) == sum(v[q]*y[i,d,q] for q in Q) for i in I for d in D)
model.addConstrs(sum(y[i,d,q] for q in Q) == 1 for i in I for d in D)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


In [19]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 470 rows, 2843370 columns and 56362466 nonzeros
Model fingerprint: 0x28e93a15
Model has 100 SOS constraints
Variable types: 600 continuous, 2842770 integer (2842770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve removed 9 rows and 162 columns (presolve time = 11s) ...
Presolve removed 9 rows and 162 columns (presolve time = 15s) ...
Presolve removed 9 rows and 162 columns (presolve time = 20s) ...
Presolve removed 9 rows and 162 columns (presolve 

In [20]:
for t in T:
  for h in H_t[t]:
    if alpha[t,h].X == 1:
      print(t, a[t-1][h-1])
      # print(f"Time Period : {t} uses schedule {h}")


1 (1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0)
2 (0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0)
3 (0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0)
6 (1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1)
7 (0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1)
8 (0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0)
9 (1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0)
10 (0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0)
11 (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1)
12 (0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0)
13 (0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0)
16 (0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1)
18 (0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0)
19 (0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0)
22 (0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0)
23 (0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0

In [21]:
print(sum(alpha[1,h].X for h in H_t[1]))

1.0
